In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import math
from tabulate import tabulate

pd.set_option('display.max_columns', 50)

## Functions needed

In [2]:
# For Newton-Raphson method

def nrfn(x): return math.log(x) - 6.949 + (3.46*(10**-6))*(x**2)
def derivative(x): return (1/x) + (6.92*(10**-6))*x
#Functions for NR for multiple variables
def nrfn1(x,y,z): return x**3 + 3*y**2 -24.6
def nrfn2(x,y,z): return x**2 + 2*y + 2
def nrfn3(x,y,z): return z
def J(x,y,z): return np.array([[3*x**2,6*y,0],
                               [2*x,2,0],
                               [0,0,1]])

### Newton Raphson algorithm

#### Single Variable optimization

In [4]:
# Newton Raphson Method

ITERATION = 6
INITIAL = 1000

iterNo = np.arange(1,ITERATION+1)
Xi = []
fXi = []
dervXi = []
Xi1 = []
norm = []

curr = INITIAL
for _ in iterNo:
    Xi.append(curr)
    
    fXi.append(nrfn(curr))
    dervXi.append(derivative(curr))
    
    curr = curr - fXi[-1]/dervXi[-1]
    
    Xi1.append(curr)
    
    norm.append((Xi1[-1] - Xi[-1])**2)
    
df = pd.DataFrame(np.array([iterNo,Xi,fXi,dervXi,Xi1,norm]).T, columns=['IterNo','Xi','f(Xi)','derv(Xi)','Xi+1','Norm'])
df.set_index('IterNo',inplace=True)
df

,Xi,f(Xi),derv(Xi),Xi+1,Norm
IterNo,,,,,
1.0,1000.000000,3.418755e+00,0.007920,568.338980,1.863312e+05
2.0,568.338980,5.113299e-01,0.005692,478.512519,8.068793e+03
3.0,478.512519,1.393321e-02,0.005401,475.932828,6.654810e+00
4.0,475.932828,8.441414e-06,0.005395,475.931263,2.448574e-06
5.0,475.931263,3.066880e-12,0.005395,475.931263,3.231821e-19
6.0,475.931263,0.000000e+00,0.005395,475.931263,0.000000e+00


### Multiple Variable optimization - 3 variables

In [5]:
ITERATION = 5
INITIAL = (1,-1,0)

iterNo = np.arange(1,ITERATION+1)
Xi = []
Xi1 = []
Ds = []
Js = []
Delta = []
Err = []

curr = np.array(INITIAL)
for _ in iterNo:
    Xi.append(curr)
    Js.append(J(*curr))
    
    Ds.append([-nrfn1(*curr),-nrfn2(*curr),-nrfn3(*curr)])
    delta = np.matmul(np.linalg.inv(Js[-1]),Ds[-1])
    Delta.append(delta)
    
    Xi1.append(curr+delta)
    Err.append(sum(np.nan_to_num((delta/curr)**2)))
    curr = Xi1[-1]

df = pd.DataFrame({'iterNo':iterNo,
                   'Xi':np.array(Xi)[:,0],'Yi':np.array(Xi)[:,1],'Zi':np.array(Xi)[:,2],
                   'f1':-np.array(Ds)[:,0],'f2':-np.array(Ds)[:,1],'f3':-np.array(Ds)[:,2],
                   'df1/dx1':np.array(Js)[:,0][:,0],'df1/dx2':np.array(Js)[:,0][:,1],'df1/dx3':np.array(Js)[:,0][:,2],
                   'df2/dx1':np.array(Js)[:,1][:,0],'df2/dx2':np.array(Js)[:,1][:,1],'df2/dx3':np.array(Js)[:,1][:,2],
                   'df3/dx1':np.array(Js)[:,2][:,0],'df3/dx2':np.array(Js)[:,2][:,1],'df3/dx3':np.array(Js)[:,2][:,2],
                   'del1':np.array(Delta)[:,0],'del2':np.array(Delta)[:,1],'del3':np.array(Delta)[:,2],
                   'Xi+1':np.array(Xi1)[:,0],'Yi+1':np.array(Xi1)[:,1],'Zi+1':np.array(Xi1)[:,2],
                   'Error':Err})
df.set_index('iterNo',inplace=True)
#df.drop(columns=['Zi','f3','df1/dx3','df2/dx3','df3/dx1','df3/dx2','df3/dx3','del3','Zi+1'])
df

/tmp/ipykernel_6242/1090177868.py:22: RuntimeWarning: invalid value encountered in true_divide
  Err.append(sum(np.nan_to_num((delta/curr)**2)))


,Xi,Yi,Zi,f1,f2,f3,df1/dx1,df1/dx2,df1/dx3,df2/dx1,df2/dx2,df2/dx3,df3/dx1,df3/dx2,df3/dx3,del1,del2,del3,Xi+1,Yi+1,Zi+1,Error
iterNo,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,-1.000000,0.0,-20.600000,1.000000,-0.0,3.000000,-6.000000,0.0,2.000000,2.0,0.0,0.0,0.0,1.0,1.955556,-2.455556,0.0,2.955556,-3.455556,0.0,9.853951e+00
2,2.955556,-3.455556,0.0,37.040283,3.824198,0.0,26.205926,-20.733333,0.0,5.911111,2.0,0.0,0.0,0.0,1.0,-0.876550,0.678593,0.0,2.079006,-2.776963,0.0,1.265219e-01
3,2.079006,-2.776963,0.0,7.520585,0.768339,0.0,12.966796,-16.661778,0.0,4.158012,2.0,0.0,0.0,0.0,1.0,-0.292428,0.223790,0.0,1.786578,-2.553173,0.0,2.627896e-02
4,1.786578,-2.553173,0.0,0.658591,0.085514,0.0,9.575583,-15.319040,0.0,3.573156,2.0,0.0,0.0,0.0,1.0,-0.035556,0.020766,0.0,1.751022,-2.532407,0.0,4.622335e-04
5,1.751022,-2.532407,0.0,0.008025,0.001264,0.0,9.198234,-15.194442,0.0,3.502044,2.0,0.0,0.0,0.0,1.0,-0.000492,0.000230,0.0,1.750530,-2.532177,0.0,8.732529e-08
